In [93]:
import tensorflow as tf
from einops import rearrange

tf.random.set_seed(123)

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [105]:
# Fixed Attention Test

def get_matrix(a,b):
    return tf.random.normal((a,b))

W_1 = get_matrix(5, 1)
W_2 = get_matrix(5, 1)
W_3 = get_matrix(5, 1)
W_4 = get_matrix(5, 1)

input = tf.random.normal((1,1))


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.9684915]], dtype=float32)>

In [107]:
W_1@input

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.367952 ],
       [-0.6621656],
       [ 2.15393  ],
       [ 0.6223257],
       [-0.893998 ]], dtype=float32)>

In [115]:
tf.concat([tf.matmul(W_1, input), tf.matmul(W_2, input), tf.matmul(W_3, input)], axis=1)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[ 0.367952  ,  0.47415724,  0.7760082 ],
       [-0.6621656 , -0.91501427, -1.0060192 ],
       [ 2.15393   ,  0.48954043, -0.25503266],
       [ 0.6223257 ,  0.00335142,  1.0305272 ],
       [-0.893998  , -1.2598883 ,  2.704208  ]], dtype=float32)>

In [95]:

class ESN(tf.keras.layers.Layer):
    def __init__(self, reservoir_size=100, spectral_radius=1.0, connectivity_rate=1.0, activation="tanh"):
        
        super(ESN, self).__init__()
        self.reservoir_size = reservoir_size
        self.connectivity_rate = connectivity_rate
        self.spectral_radius = spectral_radius
        self.activation = self.activation_fn(activation)


    def build(self, input_shape):
        self.state = self.add_weight(shape=)


        self.state = tf.zeros((self.reservoir_size, 1))
        self.W_in = tf.random.uniform((self.reservoir_size, self.input_size), minval=-1, maxval=1)

        self.W_out = None

        ## Initializing Reservoir Weights according to "Re-visiting the echo state property"(2012)
        ##
        ## Initialize a random matrix and induce sparsity.
        self.W_res = tf.random.normal((reservoir_size, reservoir_size))
        self.W_res = tf.where(tf.random.normal(self.W_res.shape) > self.connectivity_rate, tf.zeros_like(self.W_res), self.W_res)


        ## Scale the matrix based on user defined spectral radius.
        current_spectral_radius = tf.reduce_max(tf.abs(tf.linalg.eigvals(self.W_res)))
        self.W_res = self.W_res * (self.spectral_radius / current_spectral_radius)        

        self.all_states = [self.state]

    
    @staticmethod
    def activation_fn(x):
         
        activation_keys = ["sigmoid", "relu", "tanh"]

        if x in activation_keys:
              if x == "tanh":
                   return tf.keras.activations.tanh
              elif x == "relu":
                   return tf.keras.activations.relu()
              elif x == "sigmoid":
                   return tf.keras.activations.sigmoid()
            
        else:
            raise ValueError(f"Activation {x} does not exists")
        

    def fit(self, X_train, y_train):
        

        state_collection_matrix = tf.zeros((self.input_size + self.reservoir_size, 1))
        # self.state = np.zeros((self.reservoir_size, 1))

        ## Calculate state of reservoirs per time step
        for i in range(X_train.shape[0]-1):

            

            input = rearrange(X_train[i], 'c -> c 1')
            
            # print(input.dtype, self.W_in.dtype)
            input_product = self.W_in@input
            state_product = self.W_res@self.state
            self.state = self.activation(input_product + state_product)
            state_collection_matrix= tf.concat([state_collection_matrix, tf.concat([self.state, input], axis=0)], axis=1)

            self.all_states.append(self.state)

        ## Update W_out
        mat1 = tf.transpose(state_collection_matrix)[self.washout:,:]
        self.W_out = tf.transpose(tf.linalg.lstsq(matrix=mat1, rhs=y_train[self.washout:,:], l2_regularizer=0.01))

        
        
    def predict(self, X_test):
            prediction = tf.zeros((self.output_size,1))
            for i in range(X_test.shape[0]- 1):
                input = rearrange(X_test[i], 'c -> c 1')
                
                input_product = self.W_in@input
                state_product = self.W_res@self.state
                self.state = self.activation(input_product + state_product)
                concat_matrix= tf.concat([self.state, input], axis=0)
                # print(self.W_out.shape, concat_matrix.shape)
                pred =  self.W_out@concat_matrix
                prediction = tf.concat([prediction, pred], axis=1)

                self.all_states.append(self.state)
            
            prediction = rearrange(prediction, 'c r -> r c')
            if self.output_size == self.input_size:
                return prediction[1:,:]
            else:
                return prediction

In [96]:
time_series = AirPassengersDataset().load()
X = time_series.values()
X_train, X_test = train_test_split(X, test_size=0.2, shuffle=False)

sc = MinMaxScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [97]:
X_train_std = tf.convert_to_tensor(X_train_std, dtype=tf.float32)
X_test_std = tf.convert_to_tensor(X_test_std, dtype=tf.float32)

In [98]:
esn = ESN(reservoir_size=20, input_size=1, output_size=1, spectral_radius=0.7, connectivity_rate=0.8, washout=1)

In [99]:
esn.summary()

Model: "esn_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [100]:
esn.fit(X_train_std, X_train_std)

In [101]:
esn.predict(X_test_std)

<tf.Tensor: shape=(28, 1), dtype=float32, numpy=
array([[1.0219061 ],
       [0.6251821 ],
       [0.63051164],
       [0.4909718 ],
       [0.7011042 ],
       [0.7003457 ],
       [0.6162143 ],
       [0.79905677],
       [0.6953214 ],
       [0.83498377],
       [0.9287133 ],
       [1.121824  ],
       [1.0545398 ],
       [0.7571722 ],
       [0.7168091 ],
       [0.6399275 ],
       [0.85743266],
       [0.8104043 ],
       [0.7187937 ],
       [0.800771  ],
       [0.9060784 ],
       [0.9192144 ],
       [1.0868392 ],
       [1.2482482 ],
       [1.1130052 ],
       [0.8471237 ],
       [0.8318977 ],
       [0.67082405]], dtype=float32)>

In [102]:
test_values= X_test_std[1:]
test_values.shape
predictions = esn.predict(X_test_std)
predictions.shape
print("MSE:", np.sqrt(mean_squared_error(test_values, predictions)))
print("MAE:", mean_absolute_error(test_values, predictions))
print("MAPE:",mean_absolute_percentage_error(test_values, predictions))

MSE: 0.13514134
MAE: 0.111692026
MAPE: 0.1327499
